In [1]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 73.6 MB/s eta 0:00:00


In [2]:
import os, json, fitz, torch
from PIL import Image
from datasets import load_dataset
from transformers import (
    AutoProcessor, LlavaOnevisionForConditionalGeneration,
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments, DataCollatorForLanguageModeling,
    pipeline
)

2025-08-14 12:31:31.925623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755174692.123800      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755174692.181866      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
PDF_DIR = "/kaggle/input/eira-0-1-datasets"
IMG_MODEL_DIR = "/kaggle/input/eira-0.2-imageprocessing/transformers/default/1"
TXT_MODEL_DIR = "/kaggle/input/eira-0.2-textgeneration/transformers/default/1"

IMAGE_SIZE = (384, 384)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
processor = AutoProcessor.from_pretrained(IMG_MODEL_DIR)
vl_model = LlavaOnevisionForConditionalGeneration.from_pretrained(
    IMG_MODEL_DIR, torch_dtype=torch.float16,
    low_cpu_mem_usage=True, device_map={"": 0}  
)
vl_model.eval()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


LlavaOnevisionForConditionalGeneration(
  (vision_tower): SiglipVisionModel(
    (vision_model): SiglipVisionTransformer(
      (embeddings): SiglipVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(729, 1152)
      )
      (encoder): SiglipEncoder(
        (layers): ModuleList(
          (0-25): 26 x SiglipEncoderLayer(
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (self_attn): SiglipAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SiglipMLP(
     

In [5]:
page_data = {}
os.makedirs("page_images", exist_ok=True)

for fn in os.listdir(PDF_DIR):
    if not fn.lower().endswith(".pdf"):
        continue
    
    pdf_name = fn[:-4]
    page_data[pdf_name] = {}
    
    doc = fitz.open(os.path.join(PDF_DIR, fn))
    
    for pageno, page in enumerate(doc, start=1):
        print(f"Processing {pdf_name}, Page {pageno}")
        
        # Extract text from page
        page_text = page.get_text().strip()
        
        # Render page to image
        pix = page.get_pixmap(dpi=200)
        img_path = f"page_images/{pdf_name}_p{pageno}.png"
        pix.save(img_path)
        
        # Load image (skip OCR)
        im = Image.open(img_path).convert("RGB").resize(IMAGE_SIZE)
        
        # Initialize page data
        page_info = {
            "pdf_name": pdf_name,
            "page_number": pageno,
            "image_path": img_path,
            "extracted_text": page_text,
            "has_diagram": False,
            "vision_qa": None
        }
        
        # Check if page has text content to process vision
        if len(page_text) >= 20:  # Use extracted text instead of OCR
            question = f"Explain the content shown in this biology textbook page focusing on any diagrams, figures, or visual elements."
            user_msg = {
                "role": "user", 
                "content": [
                    {"type": "text", "text": question},
                    {"type": "image"}
                ]
            }
            
            prompt = processor.apply_chat_template([user_msg], add_generation_prompt=True)
            inputs = processor(text=prompt, images=im, return_tensors="pt")
            
            # Ensure all inputs are on the same device as the model
            inputs = {k: v.to(vl_model.device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
            
            with torch.cuda.amp.autocast():  # Use automatic mixed precision
                out_ids = vl_model.generate(
                    **inputs,
                    max_new_tokens=150,
                    pad_token_id=processor.tokenizer.eos_token_id
                )
            gen_txt = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
            answer = gen_txt.split("assistant")[-1].strip()
            
            page_info["has_diagram"] = True
            page_info["vision_qa"] = {
                "question": question,
                "answer": answer
            }
        
        page_data[pdf_name][pageno] = page_info
    
    doc.close()

Processing ImgSample, Page 1


/tmp/ipykernel_19/3200619703.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Use automatic mixed precision


Processing ImgSample, Page 2
Processing ImgSample, Page 3
Processing ImgSample, Page 4
Processing ImgSample, Page 5
Processing ImgSample, Page 6
Processing ImgSample, Page 7
Processing ImgSample, Page 8
Processing ImgSample, Page 9
Processing ImgSample, Page 10
Processing ImgSample, Page 11
Processing ImgSample, Page 12
Processing ImgSample, Page 13
Processing ImgSample, Page 14
Processing ImgSample, Page 15
Processing ImgSample, Page 16
Processing ImgSample, Page 17
Processing ImgSample, Page 18
Processing ImgSample, Page 19
Processing ImgSample, Page 20
Processing ImgSample, Page 21
Processing ImgSample, Page 22
Processing ImgSample, Page 23
Processing ImgSample, Page 24
Processing ImgSample, Page 25
Processing ImgSample, Page 26
Processing ImgSample, Page 27


In [6]:
with open("page_mapped_data.json", "w", encoding="utf-8") as f:
    json.dump(page_data, f, indent=2)

In [7]:
def create_enhanced_training_data():
    enhanced_samples = []
    
    for pdf_name, pages in page_data.items():
        for page_num, page_info in pages.items():
            context_prefix = f"[PDF: {pdf_name}, Page: {page_num}] "
            
            # Text-based sample
            if page_info["extracted_text"]:
                text_content = context_prefix + page_info["extracted_text"]
                enhanced_samples.append(text_content)
            
            # Vision-based sample
            if page_info["has_diagram"] and page_info["vision_qa"]:
                vision_content = context_prefix + f"Q: {page_info['vision_qa']['question']} A: {page_info['vision_qa']['answer']}"
                enhanced_samples.append(vision_content)
            
            # Combined multimodal sample
            if page_info["extracted_text"] and page_info["has_diagram"]:
                combined_content = context_prefix + f"Text: {page_info['extracted_text']} Visual: {page_info['vision_qa']['answer']}"
                enhanced_samples.append(combined_content)
    
    return enhanced_samples

In [8]:
training_samples = create_enhanced_training_data()

In [9]:
with open("enhanced_combined_train.txt", "w", encoding="utf-8") as f:
    for sample in training_samples:
        f.write(sample + "\n\n")

In [10]:
tokenizer = AutoTokenizer.from_pretrained(TXT_MODEL_DIR, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(TXT_MODEL_DIR, trust_remote_code=True)
model.config.sliding_window = None
model.to(DEVICE)

# Load and tokenize dataset
dataset = load_dataset("text", data_files={"train": "enhanced_combined_train.txt"})["train"]

def tokenize_fn(examples):
    return tokenizer(
        examples["text"], 
        truncation=True, 
        padding="max_length", 
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1904 [00:00<?, ? examples/s]

In [11]:
training_args = TrainingArguments(
    output_dir="./multimodal",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=20,
    save_strategy="epoch",
    warmup_steps=100,
    weight_decay=0.01,
    report_to="none"
)

In [12]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

In [13]:
trainer.train()

Step,Training Loss
20,1.752300
40,1.694800
60,1.313100
80,1.428900
100,1.458100
120,1.231100
140,1.363000
160,1.282400
180,1.231200
200,1.059800


TrainOutput(global_step=714, training_loss=0.8109090368286902, metrics={'train_runtime': 1651.0156, 'train_samples_per_second': 3.46, 'train_steps_per_second': 0.432, 'total_flos': 6280132656365568.0, 'train_loss': 0.8109090368286902, 'epoch': 3.0})

In [14]:
model.save_pretrained("./multimodal")
tokenizer.save_pretrained("./multimodal")

('./multimodal/tokenizer_config.json',
 './multimodal/special_tokens_map.json',
 './multimodal/vocab.json',
 './multimodal/merges.txt',
 './multimodal/added_tokens.json',
 './multimodal/tokenizer.json')

In [15]:
def print_trainable_params(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total Parameters:    {total:,}")
    print(f"Trainable Parameters: {trainable:,} ({100 * trainable / total:.2f}%)")

print_trainable_params(model)

Total Parameters:    494,032,768
Trainable Parameters: 494,032,768 (100.00%)


In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_dir = "./multimodal"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForCausalLM.from_pretrained(model_dir)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Questions to test
eval_questions = [
    "What are the symptoms of bronchiectasis and how do I get myself cured for the same?",
    "List me at least 10 medications to cure myself from asthma.",
    "I am suffering from frequent chills, cold, running nose, and fatigue. This happened after a mosquito bite in the evening yesterday. What do you think I might be suffering from?",
    "A female anopheles mosquito bit me and what will I be suffering from now? Now tell me preventive measures to avoid such mosquitoes in my house and also list me medications/drugs to cure myself from the disease /disorder."
]

# Run evaluation
for idx, question in enumerate(eval_questions, 1):
    prompt = f"Question: {question}\nAnswer:"
    output = pipe(prompt, max_new_tokens=100, do_sample=False, temperature=0.0,
                  pad_token_id=tokenizer.eos_token_id)
    answer = output[0]['generated_text'].split("Answer", 1)[-1].strip(": ").strip()
    print(f"\n🧠 Q{idx}: {question}\n🔍 Answer: {answer}")


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



🧠 Q1: What are the symptoms of bronchiectasis and how do I get myself cured for the same?
🔍 Answer: The most common lower respiratory tract infection is pneumonia. The top three most common lower respiratory tract infections are: bacterial pneumonia, viral pneumonia, and fungal pneumonia. The top three most common lower respiratory tract infections are: bacterial pneumonia, viral pneumonia, and fungal pneumonia. The top three most common lower respiratory tract infections are: bacterial pneumonia, viral pneumonia, and fungal pneumonia. The top three most common lower respiratory tract infections are: bacterial pneumonia, viral pneumonia, and fungal pneumonia. The top three most common lower

🧠 Q2: List me at least 10 medications to cure myself from asthma.
🔍 Answer: 1. Ciliated nasal cavity, which causes swelling and inflammation of the nasal cavity. 2. Nasal polyps, which are large, immature, and non-ciliated growth of tissue that lie outside of the nasal cavity. 3. Nasal Cavity Cavi